# Training a language model with fastai
- train a language model with curated dataset IMDB using pre-trained model AWD_LSTM


In [1]:
#hide
!pip install -Uqq fastbook
import fastbook
fastbook.setup_book()

In [2]:
#hide
from fastbook import *
from fastai.text.all import *

In [3]:
modifier = 'may15'

# Training a language model
- take a pretrained model and train it some more using the IMDB dataset

In [4]:
%%time
# create path object
path = untar_data(URLs.IMDB)
path.ls()

CPU times: user 3.9 ms, sys: 0 ns, total: 3.9 ms
Wall time: 9.7 ms


(#7) [Path('/storage/data/imdb/train'),Path('/storage/data/imdb/imdb.vocab'),Path('/storage/data/imdb/tmp_lm'),Path('/storage/data/imdb/unsup'),Path('/storage/data/imdb/tmp_clas'),Path('/storage/data/imdb/README'),Path('/storage/data/imdb/test')]

In [5]:
%%time
# create TextDataLoaders object
get_imdb = partial(get_text_files, folders=['train', 'test', 'unsup'])
dls = TextDataLoaders.from_folder(path, valid = 'test', is_lm=True, bs=16)

CPU times: user 4.62 s, sys: 1.93 s, total: 6.54 s
Wall time: 8.95 s


In [6]:
dls.show_batch(max_n=2)

,text,text_
0,xxbos xxmaj this movie was the most out of line and liberally fed movie i have ever seen in my life . ( besides xxmaj farenheit 9 / 11 ) . xxmaj all of the information was only supported on the opinion of xxup five scientists while 80 % of the a xxrep 3 s xxunk xxmaj press highly criticize the science promoted be xxmaj gore . xxmaj global xxmaj warming is,xxmaj this movie was the most out of line and liberally fed movie i have ever seen in my life . ( besides xxmaj farenheit 9 / 11 ) . xxmaj all of the information was only supported on the opinion of xxup five scientists while 80 % of the a xxrep 3 s xxunk xxmaj press highly criticize the science promoted be xxmaj gore . xxmaj global xxmaj warming is a
1,"that was the crucible for so much of our modern civilization . xxmaj it teaches us much about ourselves as individuals and as social and political creatures . xxmaj euripides questions the value of war and patriotism when measured against the simple virtues of family and love , and reflects on woman 's vulnerable position in a world of manly violence . xxmaj in his adaptation of xxmaj euripides ' tragedy ,","was the crucible for so much of our modern civilization . xxmaj it teaches us much about ourselves as individuals and as social and political creatures . xxmaj euripides questions the value of war and patriotism when measured against the simple virtues of family and love , and reflects on woman 's vulnerable position in a world of manly violence . xxmaj in his adaptation of xxmaj euripides ' tragedy , xxmaj"


In [7]:
%%time
# define and train model
learn = language_model_learner(dls,AWD_LSTM,metrics=accuracy).to_fp16()
learn.fine_tune(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,4.745757,4.347702,0.266524,30:27


epoch,train_loss,valid_loss,accuracy,time
0,4.260123,4.043935,0.296656,30:54


CPU times: user 45min 17s, sys: 13min 59s, total: 59min 17s
Wall time: 1h 1min 36s


In [8]:
# get prediction
# preds = learn.predict('The star is', n_words=20)
learn.predict("what comes next", n_words=20)

'what comes next ? Why would anyone have done anything better yet who watched this then ? The acting is ridiculously'

In [9]:
keep_path = learn.path

In [18]:
# workaround to make path writeable
learn.path = Path('/notebooks/temp')

In [19]:
learn.path

Path('/notebooks/temp')

In [12]:
learn.model_dir

'models'

In [13]:
learn.save('lm_'+modifier)

Path('/notebooks/temp/models/lm_may15.pth')

In [14]:
# workaround to save encoder - need to do this to later load encoder for classifier
learn.save_encoder('ft_'+modifier)

In [20]:
learn.export('lm_model_'+modifier+'.pkl')